 # Надо:
* Считать данные из текстового файла
* Переделать в numpy array(2 размерности)
* **ОБЯЗАТЕЛЬНО** нагенерить данных для обучения
* Разбить на выборки
* Обучить модель
* Перевести предсказания в пары

In [1]:
import numpy as np
import pandas as ps

In [8]:
dtt = open('userset.txt', 'r')
dt = dtt.readlines()
dtt.close()
dat = [] # Будущие данные о пользователях
for usr in dt:
    dat.append(usr.split(','))  # ВАЖНО! разделитель данных -- ЗАПЯТАЯ

#''' Данные о пользователе: [id, nickname, room_number, direction, (sports...)]
#                           [0,  kllllevo, 13,           1,         0,1,0,0]
#'''
dat

[['1', 'Екатерина Самохина', '16', '1', '1', '0', '0\n'],
 ['2', 'Даниил Царьков', '6', '2', '0', '0', '0\n'],
 ['3', 'Илья Николаев', '6', '2', '0', '0', '0\n'],
 ['4', 'Григорий Морозов', '14', '4', '1', '0', '0\n'],
 ['5', 'Автономова Ксения', '22', '3', '0', '0', '0\n'],
 ['6', 'Пушкарев Святослав', '20', '3', '0', '0', '0\n']]

In [3]:
data = {}
for usr in dat:
    data[usr[0]] = list(usr[1:])

#''' Данные о пользователях: {'id':[Данные о пользователе, кроме id]} 
#'''
#data

In [4]:
drata = []
for i in list(data.keys()):
    a = list(data.keys())
    a.remove(i)
    drata.append([val for val in a])

#''' Данные вида [another users id] -- this for user with id=0
#                [another users id] -- for id=1
#                [another users id] -- for id=2
#'''
#drata

In [5]:
final_data = []
nmb = 1 # id юзера(см. ниже)
for lst in drata:
    top_ids= [] #здесь хранятся id
    with_ids = [] # тут -- число совпадений
    top = []
    for u in lst:
        u_data = np.array(data[u][1:])# Вся дата того, кого проверяем, кроме nickname
        usr_data = np.array(data[str(nmb)][1:])# Вся дата того, ДЛЯ кого проверяем, кроме nickname
        with_ids.append(len(np.where(u_data == usr_data)[0]))
        top_ids.append(u)
    args_by_sort = np.argsort(np.array(with_ids))
    #print('top_ids',top_ids)
    #print('with_ids',with_ids)
    #print('args_by_sort',args_by_sort)
    for arg in args_by_sort:
        top.append(top_ids[arg])
    final_data.append(top)
    nmb += 1
#'''Вот это б*ть то, ради чего мы все тут собрались. Данная клетка воплощает идею о том,
#Чтобы составить все возможные пары киллер-жертва, и отсортировать потенциальных жертв по
#кол-ву совпадений по признакам. Выхлоп -- внизу
#'''
#print(final_data[0])

In [6]:
weighted_data = []
for lst in final_data:#Не забыть nmb += 1
    weighted = []
    cloned_final = list(final_data)
    cloned_final.remove(lst)
    for u in lst:   
        weight = 0
        for sub_lst in cloned_final:
            for sub_u in sub_lst:
                if sub_lst.index(sub_u) <= lst.index(u) and u == sub_u:
                    weight += 1 + int(lst.index(u)) - int(sub_lst.index(sub_u))
        weighted.append(weight)
    weighted_data.append(weighted)
#''' Данная чудо-ячейка возвращает нам матрицу с "весами" id ,где каждому элементу этого массива
#сопоставляется id из final_data
#'''
#weighted_data

In [7]:
pair = []
for n in range(0, len(final_data)):
    ind = np.argmin(weighted_data[n])
    cdel = int(final_data[n][ind]) - 1
    for j in range(len(final_data[cdel])):
        if final_data[cdel][j] == str(n + 1):
            del weighted_data[cdel][j]
            del final_data[cdel][j]
            break
    pair.append(cdel + 1)
    for i in range(len(final_data)):
        if i != n and i != cdel:
            for j in range(0, len(final_data[i])):
                if final_data[i][j] == str(cdel + 1):
                    del final_data[i][j]
                    del weighted_data[i][j]
                    break
print(pair)

ValueError: attempt to get argmin of an empty sequence

In [100]:
fil = open('pairs.txt', 'w')
for i in pair:
    fil.write(data[i[0]][0]+','+data[i[1]][0]+"\n")
fil.close()